In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

zip_path = '/content/drive/My Drive/MenuMakanan.zip'

import zipfile

extracted_path = '/content/MenuMakanan'  # Ganti dengan path tempat Anda ingin mengekstrak file ZIP

# Ekstraksi file ZIP
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_path)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, LSTM, Dense, Reshape, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Path ke dataset yang telah dimuat
train_path = '/content/MenuMakanan/MenuMakanan/training'
validation_path = '/content/MenuMakanan/MenuMakanan/validation'
test_path = '/content/MenuMakanan/MenuMakanan/test'

# Augmentasi dan prapemrosesan data menggunakan ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

img_height = 64
img_width = 64
img_channels = 3  # Ubah ke 1 jika menggunakan citra grayscale
num_classes = 2  # Sesuaikan dengan jumlah kategori/bahasa isyarat yang ada dalam dataset
num_epochs = 50
batch_size = 32  # Atur sesuai dengan kebutuhan Anda

train_generator = train_datagen.flow_from_directory(
        train_path,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='categorical')

validation_generator = validation_datagen.flow_from_directory(
        validation_path,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
        test_path,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='categorical')

# CNN model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, img_channels)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())

# LSTM model
model.add(Dense(64, activation='relu'))
model.add(Reshape((1, -1)))
model.add(Dropout(0.2))
model.add(LSTM(128))

# Output layer
model.add(Dense(num_classes, activation='softmax'))

# Compile model
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(train_generator, epochs=num_epochs, validation_data=validation_generator)

model.save("content/model.h5")

# Evaluate the model
accuracy = model.evaluate(test_generator)
print(f"Test Accuracy: {accuracy}")

Found 602 images belonging to 2 classes.
Found 154 images belonging to 2 classes.
Found 59 images belonging to 2 classes.
Epoch 1/50
19/19 [==============================] - 10s 397ms/step - loss: 0.6765 - accuracy: 0.5980 - val_loss: 0.6954 - val_accuracy: 0.4935
Epoch 2/50
19/19 [==============================] - 5s 254ms/step - loss: 0.6605 - accuracy: 0.6130 - val_loss: 0.6918 - val_accuracy: 0.5065
Epoch 3/50
19/19 [==============================] - 7s 352ms/step - loss: 0.6180 - accuracy: 0.6611 - val_loss: 0.6400 - val_accuracy: 0.6364
Epoch 4/50
19/19 [==============================] - 5s 260ms/step - loss: 0.5250 - accuracy: 0.7558 - val_loss: 0.6305 - val_accuracy: 0.6623
Epoch 5/50
19/19 [==============================] - 7s 369ms/step - loss: 0.4487 - accuracy: 0.8040 - val_loss: 0.6621 - val_accuracy: 0.6688
Epoch 6/50
19/19 [==============================] - 5s 242ms/step - loss: 0.4182 - accuracy: 0.8256 - val_loss: 0.4630 - val_accuracy: 0.7857
Epoch 7/50
19/19 [=======

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


2/2 [==============================] - 0s 48ms/step - loss: 0.7653 - accuracy: 0.8983
Test Accuracy: [0.7653478980064392, 0.8983050584793091]


In [ ]:
import tensorflow.lite as tf_lite

# Assuming you have a Keras model named 'model' already defined

# Convert the model to TFLite
converter = tf_lite.TFLiteConverter.from_keras_model(model)
converter.target_spec.supported_ops = [tf_lite.OpsSet.TFLITE_BUILTINS, tf_lite.OpsSet.SELECT_TF_OPS]
converter.experimental_new_converter = True

# Disable experimental lower tensor list ops
converter._experimental_lower_tensor_list_ops = False

tflite_model = converter.convert()

# Save the TFLite model
tflite_model_path = "content/model.tflite"
with open(tflite_model_path, "wb") as f:
    f.write(tflite_model)

print("TFLite model saved successfully at:", tflite_model_path)

TFLite model saved successfully at: content/model.tflite


In [ ]:

from tensorflow.keras.preprocessing import image
import numpy as np

# Path ke gambar yang ingin diuji
test_image_path = '/content/crispy-comte-cheesburgers-FT-RECIPE0921-6166c6552b7148e8a8561f7765ddf20b.jpg'  # Ganti dengan path gambar Anda

# Fungsi untuk memprediksi gambar
def predict_image(model, img_path, labels):
    img = image.load_img(img_path, target_size=(img_height, img_width))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = img_array / 255.0  # Normalisasi seperti pada proses training

    # Melakukan prediksi
    prediction = model.predict(img_array)
    predicted_class = np.argmax(prediction, axis=1)
    predicted_label = labels[predicted_class[0]]
    return predicted_label

# List label sesuai dengan kelas numerik dari model
labels = ['burger','french fries']

# Melakukan prediksi untuk gambar tertentu
predicted_class = predict_image(model, test_image_path, labels)
print(f'Predicted class: {predicted_class}')

1/1 [==============================] - 0s 27ms/step
Predicted class: burger
